In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler


In [2]:
# Read the dataset
data = pd.read_csv('output_Cadets.csv')

# Encode categorical columns using LabelEncoder
categorical_columns = ['Source UUID', 'Source type', 'Destination UUID', 'Destination type', 'Edge type']
encoder = LabelEncoder()

for col in categorical_columns:
    data[col] = encoder.fit_transform(data[col])

# Split the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode the target variable (Label)
y = LabelEncoder().fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Create a neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model with a learning rate of 0.001
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=1e-6)
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks=[reduce_lr])


Epoch 1/30
97466/97466 [==============================] - 239s 2ms/step - loss: 0.1093 - accuracy: 0.9485 - val_loss: 0.0776 - val_accuracy: 0.9617
Epoch 2/30
97466/97466 [==============================] - 257s 3ms/step - loss: 0.0867 - accuracy: 0.9588 - val_loss: 0.0657 - val_accuracy: 0.9700
Epoch 3/30
97466/97466 [==============================] - 227s 2ms/step - loss: 0.0766 - accuracy: 0.9631 - val_loss: 0.0617 - val_accuracy: 0.9760
Epoch 4/30
97466/97466 [==============================] - 207s 2ms/step - loss: 0.0716 - accuracy: 0.9667 - val_loss: 0.0551 - val_accuracy: 0.9767
Epoch 5/30
97466/97466 [==============================] - 193s 2ms/step - loss: 0.0655 - accuracy: 0.9726 - val_loss: 0.0562 - val_accuracy: 0.9787
Epoch 6/30
97466/97466 [==============================] - 206s 2ms/step - loss: 0.0672 - accuracy: 0.9689 - val_loss: 0.0475 - val_accuracy: 0.9752
Epoch 7/30
97466/97466 [==============================] - 206s 2ms/step - loss: 0.0588 - accuracy: 0.9764 - val_

In [ ]:
# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Decode the predictions to 'Attack' or 'Normal'
y_pred = LabelEncoder().fit(['Attack', 'Normal']).inverse_transform(y_pred.ravel())